In [1]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.1.0
Default GPU Device: /device:GPU:0


In [2]:
BATCH_SIZE = 64
WIDTH = 200
HEIGHT = 200
CHANNEL = 3

In [3]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(WIDTH, HEIGHT, CHANNEL)))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

Using TensorFlow backend.


In [4]:
from keras.optimizers import SGD
optimizer = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

trainGen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.5,
    width_shift_range=0.1,
    height_shift_range=0.1
)
trainGenerator = trainGen.flow_from_directory('./train/', class_mode='binary', target_size=(WIDTH, HEIGHT), batch_size=BATCH_SIZE)

testGen = ImageDataGenerator(rescale=1./255)
testGenerator = testGen.flow_from_directory('./test/', class_mode='binary', target_size=(WIDTH,HEIGHT), batch_size=BATCH_SIZE)

Found 25000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
EPOCHS = 100
history = model.fit_generator(
    trainGenerator, 
    steps_per_epoch=len(trainGenerator), 
    epochs=EPOCHS, 
    verbose=1, 
    validation_data=testGenerator,
    validation_steps=len(testGenerator)
)

In [ ]:
model.save('./myModel-993.h5')